In [45]:
import gc
import time
import numpy as np
import pandas as pd
import warnings

from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GroupKFold

import os
import sys
sys.path.append(os.path.dirname(os.getcwd()))

from project_utils import kd_utils
from importlib import reload

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
from sklearn.pipeline import make_pipeline, make_union, Pipeline
warnings.filterwarnings("ignore", category=DeprecationWarning)

# data checking

In [ ]:
train.

# 19/05/2018

In [64]:
train_fs2=kd_utils.load_data('../feature_data/train_fs2.pkl')
test_fs2=kd_utils.load_data('../feature_data/test_fs2.pkl')

In [66]:
train_fs2.shape

(852885, 36)

In [31]:
train=pd.read_csv('../input/train.csv', sep='\t')
test=pd.read_csv('../input/test.csv', sep='\t')

In [57]:
uid=[]
group_kfold = GroupKFold(n_splits=5)

for train_index, val_index in group_kfold.split(train, groups=train.question_id):
    print(train_index[0:10], val_index[0:10])
    uid.append([train_index, val_index])
    
kd_utils.pickle_data('../input/indices.pkl', uid)

[0 1 2 3 4 5 6 7 8 9] [31 36 37 38 42 46 51 52 53 56]
[0 1 2 3 4 5 6 7 8 9] [ 17  19  40  48  93 105 106 118 167 168]
[0 1 2 3 4 5 6 7 8 9] [14 15 21 43 44 45 47 86 87 90]
[13 14 15 16 17 19 21 22 25 28] [0 1 2 3 4 5 6 7 8 9]
[0 1 2 3 4 5 6 7 8 9] [13 16 22 25 28 29 30 41 58 89]


In [ ]:
uid=[]
for trn_idx, val_idx in train_kf.split(train, train['is_attributed']):
    uid.append([trn_idx, val_idx])
td_utils.pickle_data('../input/indices.pkl', uid)

In [53]:
train_index[0:10]

array([ 9, 10, 11, 12, 13, 14, 15, 17, 18, 20])

In [41]:
train.head(1)

,id,question_id,subreddit,question_utc,question_text,question_score,answer_utc,answer_text,answer_score,question_dt,answer_dt
0,330435,f48a5420fa6a66ecec95365dd67bdc99,AskReddit,1512086400,"All my buddies love Pacific Rim and always talk about it. They are always telling me to watch it because ""the action is awesome"". I never really have the time though, and when I do- I usually watch something that appeals to me more.",179,1512086616,"I didn't really care for it, I thought the pacing was awful",70,2017-12-01,2017-12-01 00:03:36


In [39]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852885 entries, 0 to 852884
Data columns (total 11 columns):
id                852885 non-null int64
question_id       852885 non-null object
subreddit         852885 non-null object
question_utc      852885 non-null int64
question_text     852885 non-null object
question_score    852885 non-null int64
answer_utc        852885 non-null int64
answer_text       852885 non-null object
answer_score      852885 non-null int64
question_dt       852885 non-null datetime64[ns]
answer_dt         852885 non-null datetime64[ns]
dtypes: datetime64[ns](2), int64(5), object(4)
memory usage: 71.6+ MB


In [40]:
print(train.shape)
print(test.shape)

(852885, 11)
(663082, 10)


In [23]:
# df['CreatedDate2'] = pd.to_datetime(df['CreatedDate'], unit='ms')

sample=train.head(100).copy()


In [32]:
train['question_dt']=pd.to_datetime(train['question_utc'], unit='s')
test['question_dt']=pd.to_datetime(test['question_utc'], unit='s')
train['answer_dt']=pd.to_datetime(train['answer_utc'], unit='s')
test['answer_dt']=pd.to_datetime(test['answer_utc'], unit='s')

In [36]:
train['answer_dt']=pd.to_datetime(train['answer_utc'], unit='s')
test['answer_dt']=pd.to_datetime(test['answer_utc'], unit='s')

In [44]:
train.question_score.corr(train.answer_score)

0.09988077038939108

In [59]:
train['question_dt'].describe()

count                  852885
unique                 324012
top       2017-12-07 14:59:39
freq                      665
first     2017-12-01 00:00:00
last      2017-12-07 19:20:53
Name: question_dt, dtype: object

In [63]:
train['question_dt'].tail(5)

852880   2017-12-07 19:20:48
852881   2017-12-07 19:20:50
852882   2017-12-07 19:20:50
852883   2017-12-07 19:20:52
852884   2017-12-07 19:20:53
Name: question_dt, dtype: datetime64[ns]

In [34]:
test['question_dt'].describe()

count                  663082
unique                 281092
top       2017-12-07 16:46:38
freq                      627
first     2017-12-01 00:00:00
last      2017-12-07 19:20:54
Name: question_dt, dtype: object

In [42]:
train['answer_dt'].describe()

count                  852885
unique                 443244
top       2017-12-07 16:42:18
freq                       12
first     2017-12-01 00:00:37
last      2017-12-31 23:36:21
Name: answer_dt, dtype: object

In [43]:
test['answer_dt'].describe()

count                  663082
unique                 391992
top       2017-12-01 17:28:47
freq                       10
first     2017-12-01 00:00:45
last      2017-12-31 23:29:34
Name: answer_dt, dtype: object

In [4]:
df=pd.read_csv('../reddit-dataset/gaming_minecraft.csv')

In [ ]:
cols=['text', 'title', 'url', 'id', 'subreddit', 'meta', 'time', 'author', 
      'ups', 'downs', 'authorlinkkarma', 'authorcommentkarma', 'authorisgold']

In [7]:
df.head(5)

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,NaN,463iot,minecraft,gaming,1.455644e+09,Empoleon4625,3.0,0.0,62,250,0.0
1,1,the characters hand in first person is invisible and looks glitchy when using shaders anyone know why ps i am on tekkit legends specs i5 4690k r9 280 8gb ram,464kj4,minecraft,gaming,1.455656e+09,CookiezFort,0.0,0.0,833,3248,0.0
2,2,when in the title is supposed to be went,d02hfso,minecraft,gaming,1.455663e+09,Empoleon4625,1.0,0.0,62,250,0.0
3,3,i have an automated sword dispenser at my blaze grinder most of the swords do nt even recirculate into the hopper dropper combo at the end of my farming session i go get the blaze rods but there s like a dozen swords in the chest that collects blaze rods how can i prevent this from happening i want to somehow create a barrier but still be able to attack i ve tried fence posts near the corners but it does nt seem to work my setup http imgurcomaambrf i stand on the hopper where the dropper is aiming and the blaze are there on the hopper it s standing on in the pic on my actual farm i do nt have the glass there it s empty but as stated i did try to put up fence posts in the corner for what it s worth the blaze hopper on my factions server is actually even another block away do nt overthink that part but it s 2 back and 1 to the left away from hopper hopper still eating it though,466c3h,minecraft,gaming,1.455680e+09,Illsonmedia,2.0,0.0,389,1907,0.0
4,4,i guess either the shaders use some feature that were only added in the 18 glsl versions or more likely the shaders conflict with some other mod that changes rendering methods,d02cxvk,minecraft,gaming,1.455657e+09,Huntron,1.0,0.0,5534,3062,1.0
